# Medicotask - densenet

# Medicodataset 70-30

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os

In [3]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [4]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [34]:
PATH = "../../../data/data_generated_medicotask_70_30_v2/"
sz= 224 #64 # changed to 64 - 64X64 is good for satalite image analysis, Why? to study
arch = densenet169
bs=16

In [35]:
os.listdir(PATH)

['test',
 'keras_history',
 'pytorch_plots',
 'pytorch_gpu_info',
 'validation',
 'keras_plots',
 'pytorch_models',
 'mxnet_models',
 'keras_gpu_info',
 'tmp',
 'mxnet_history',
 'models',
 'pytorch_history',
 'mxnet_plots',
 'keras_models',
 'train']

In [36]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4, val_name='validation')
learn = ConvLearner.pretrained(arch, data)

/home/vajira/.local/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/vajira/.local/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/vajira/.local/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/vajira/.local/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/vajira/.local/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in

/home/vajira/.local/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
/home/vajira/.local/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


In [37]:
lrf=learn.lr_find()
learn.sched.plot()

  0%|          | 0/232 [00:00<?, ?it/s]


ValueError: Input dimension should be at least 3

In [ ]:
lr = 0.2

In [ ]:
learn.fit(lr, 4, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_loss()

In [ ]:
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.fit(lrs, 4, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('7_1_fastai_resnet50_medico_15_classes_v2')

In [ ]:
learn.sched.plot_loss()

In [ ]:
sz=128

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4,val_name='validation')

In [ ]:
learn.set_data(data)
learn.freeze()
learn.fit(lr, 4, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 4, cycle_len=1, cycle_mult=2)


In [ ]:
learn.save('7_1_fastai_resnet50_medico_15_classes_v2_2')

In [ ]:
learn.load('7_1_fastai_resnet50_medico_15_classes_v2_2')

In [ ]:
sz=256
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4,val_name='validation')

In [ ]:
learn.set_data(data)
learn.freeze()
learn.fit(lr, 4, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('7_1_fastai_resnet50_medico_15_classes_v2_3')

In [ ]:
learn.load('7_1_fastai_resnet50_medico_15_classes_v2_3')

In [ ]:
learn.unfreeze()
learn.fit(lrs, 5, cycle_len=1, cycle_mult=2) # changed 4 to 5 (epochs)
learn.save('7_1_fastai_resnet50_medico_15_classes_v2_last')

In [ ]:
learn.sched.plot_loss()

In [ ]:
learn.load('7_1_fastai_resnet50_medico_15_classes_v2_last')

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 4, cycle_len=1, cycle_mult=2)
learn.save('7_1_fastai_resnet50_medico_15_classes_v2_last_2')

## Analyzing results

### Confusion matrix 

In [ ]:
preds = np.argmax(probs, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]

In [ ]:
plot_confusion_matrix(cm, data.classes)

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, val_name='validation', num_workers=4)

In [ ]:
log_preds_t,y_t = learn.TTA()

In [ ]:
probs_t = np.mean(np.exp(log_preds_t),0)
accuracy_np(probs_t, y_t)

In [ ]:
preds_t = np.argmax(probs_t, axis=1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_t, preds_t)
plot_confusion_matrix(cm, data.classes)